In [1]:
# Алексеев Д.П. (DSU-4,FEML-8)
# Домашнее задание к лекции «Классификация: Логистическая регрессия и SVM» (#2).
# (по лекции №1 "Регрессионный анализ. Линейная, полиномиальная и логарифмическая регрессия"
# в качестве ДЗ были вопросы для самопроверки)

# Задание:
# Имеются данные adult.csv (см. в материалах к занятию).
# Целевой переменной является уровень дохода income (крайний правый столбец).
# Описание признаков можно найти по ссылке:
# http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html
# Вам необходимо построить модели логистической регрессии и SVM, которые предсказывают уровень дохода человека.
# Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('E:\\Netology\\Machine Learning\\Practice\\adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [5]:
# найдем колонки, в которых есть неизвестные значения (в данном датасете не нулевые, а те, в которых проставлено '?')
for i in data.columns:
    for j in data[i].unique():
        if j == '?':
            print(i)
            break

workclass
occupation
native-country


In [6]:
# обнуляем строки, в которых проставлен '?' в найденных выше колонках
# (как альтернативный вариант - можно попробовать заполнить пропуски модой, т.к. это категориальные переменные)
data = data[(data['workclass'] =='?') == 0 ]
data = data[(data['occupation'] =='?') == 0 ]
data = data[(data['native-country'] =='?') == 0 ]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
# возьмем для построения модели следующие "входящие" признаки:
selectedColumns = data[[ 'age', 'workclass', 'education', 'marital-status', 'occupation', 
                        'race', 'gender', 'native-country']]
selectedColumns

,age,workclass,education,marital-status,occupation,race,gender,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Black,Male,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,White,Male,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,White,Male,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Black,Male,United-States
5,34,Private,10th,Never-married,Other-service,White,Male,United-States
...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,White,Female,United-States
48838,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,White,Male,United-States
48839,58,Private,HS-grad,Widowed,Adm-clerical,White,Female,United-States
48840,22,Private,HS-grad,Never-married,Adm-clerical,White,Male,United-States


In [10]:
# все колонки, кроме 'age', являются категориальными переменными.
# Переведем их в значения 0 и 1 методом One-hot-encoder, добавив столбцы с соответствующими названиями
X = pd.get_dummies(selectedColumns, columns = ['workclass', 'education', 'marital-status', 
                                               'occupation', 'race', 'gender', 'native-country'])
X

,age,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,38,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48838,40,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48839,58,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
48840,22,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [11]:
data['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [12]:
# как видим, целевая переменная 'income' является категориальной, но содержит только 2 уникальных значения.
# Следовательно, её можно преобразовать в булеву переменную. 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [13]:
le.fit(data['income'])

LabelEncoder()

In [14]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [15]:
# выведем расшифровку целевой переменной 'income', чтобы не запутаться с интерпретацией результатов
le.transform( [ '<=50K', '>50K', '<=50K' ] )

array([0, 1, 0])

In [16]:
# запишем в целевую переменную 'y' преобразованный столбец 'income'
y = pd.Series(data = le.transform(data['income']))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [17]:
# Разобьем датасет на данные для обучения(70%) и проверки(30%) 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [18]:
y_test.head()

20084    0
20027    0
39505    1
39534    0
12586    0
dtype: int32

In [19]:
# Приведем данные к одному масштабу при помощи StandardScaler, т.к. в противном случае данные будут несоразмерными 
# и вылетает ошибка достижения лимита итераций:
# "C:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
# STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [20]:
# теперь создадим и обучим модель логистической регрессии:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [21]:
# получим массив с предсказаниями для тестовой выборки
predictions = model.predict(X_test)
predictions[:20]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [22]:
# получим массив с вероятностями предсказаний для тестовой выборки
model.predict_proba(X_test)

array([[0.99558899, 0.00441101],
       [0.86316177, 0.13683823],
       [0.53467485, 0.46532515],
       ...,
       [0.98527491, 0.01472509],
       [0.97974974, 0.02025026],
       [0.88907646, 0.11092354]])

In [23]:
# оценим точность предсказаний на тренировочной выборке:
model.score(X_train, y_train)

0.8265360922445112

In [24]:
# оценим точность предсказаний на тестовой выборке:
model.score(X_test, y_test)

0.829291663595489

In [25]:
# Как видим, показатели "скора" достаточно высокие (более 80%), для тестовой выборки даже чуть лучше, чем для тренировочной.
# Можно сделать вывод, что рассчитанная модель логистической регрессии получилась достаточно хорошей.

In [26]:
# Теперь построим модель, использовав метод опорных векторов (SVC)
from sklearn.svm import SVC
# используем пайплайны, запустив сперва StandardScaler, а затем непосредственно SVC
from sklearn.pipeline import make_pipeline
clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [27]:
# оценим точность предсказаний на тренировочной выборке:
clf.score(X_train, y_train)

0.8342757858158268

In [28]:
# оценим точность предсказаний на тестовой выборке:
clf.score(X_test, y_test)

0.831650328001769

In [29]:
# модель методом SVC получилась чуть лучше, чем модель логистической регрессии. Но эта разница - в пределах погрешности.